## Product Info WordCloud

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import warnings
import scipy.stats as stats
from tqdm.auto import tqdm
import seaborn as sns
from matplotlib import rc
from sklearn.preprocessing import LabelEncoder
import random
import platform
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset,  DataLoader

warnings.filterwarnings('ignore')
plt.style.use('dark_background')                # 그래프 배경 색상 변경
if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')              # 한글폰트 깨짐현상 제거
else:
    rc('font', family = 'NanumGothic')

plt.rcParams['axes.unicode_minus'] = False

%config InlineBackend.figure_format = 'retina'  # 그래프 고해상도로 출력

In [ ]:
PATH = os.getcwd() + '/data/'
df_brand = pd.read_csv(PATH + 'brand_keyword_cnt.csv')
pd_info = pd.read_csv(PATH + 'product_info.csv')
sales = pd.read_csv(PATH + 'sales.csv')
df_smp = pd.read_csv(PATH + 'sample_submission.csv')
df_train = pd.read_csv(PATH + 'train.csv')
smp = train.sample(10)

In [ ]:
train_id = list(df_train.제품)
product_id = list(pd_info.제품)

cnt = 0
no_pd = []
yes_pd = []
for t in train_id:
    if t in product_id:
        cnt += 1
        yes_pd.append(t)
    else:
        no_pd.append(t)
print(cnt)

In [ ]:
cnt = 0
no = []
yes = []
for p in product_id:
    if p in train_id:
        cnt += 1
        yes.append(p)
    else:
        no.append(p)
print(cnt)
print(no[0])

In [ ]:
cnt = 0
for y in yes:
    if y in yes_pd:
        cnt += 1
    
    else:
        pass

print(len(yes) == cnt)

In [ ]:
dic = {}
dic_info = {}

for i in range(df_train.shape[0]):
    dic[i] = df_train.loc[i, '제품']
print(len(dic.keys()))

for i in range(pd_info.shape[0]):
    dic_info[i] = pd_info.loc[i, '제품']
print(len(dic_info.keys()))

In [ ]:
dic_copy = dic.copy()
info_copy = dic_info.copy()
dic_yes = {}

for k, v in dic_copy.items():
    if v in yes:
        cnt += 1
        dic_yes[k] = v
    else:
        pass

for k, v in info_copy.items():
    if v not in yes:
        del dic_info[k]
    else:
        pass

In [ ]:
id_list = list(dic_yes.keys())
info_train = df_train.loc[id_list, :]

info_id = list(dic_info.keys())
info_df = pd_info.loc[info_id, :]

In [ ]:
df_merge = pd.merge(info_train.iloc[:, 0:6], info_df)
df_merge.head(3)

In [ ]:
major_class = list(df_merge.대분류.value_counts().keys())
major_class # 대분류 코드 리스트

In [ ]:
major_1 = df_merge[df_merge.대분류 == major_class[0]]
major_2 = df_merge[df_merge.대분류 == major_class[1]]
major_3 = df_merge[df_merge.대분류 == major_class[2]]
major_4 = df_merge[df_merge.대분류 == major_class[3]]
major_5 = df_merge[df_merge.대분류 == major_class[4]]

major_list = [major_1, major_2, major_3, major_4, major_5]

In [ ]:
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
def preprocessing(major_class):
    major_class['제품특성'] = major_class['제품특성'] \
        .replace(r'[^가-힣 ]', ' ', regex=True) \
        .replace("'", '') \
        .replace(r'\s+', ' ', regex=True) \
        .str.strip() \
        .str[:255]
    
    return major_class['제품특성']

def remove_stopwords(text):
    tokens = text.split(' ')
    stops = '특징'

    meaningful_words = [w for w in tokens if stops not in w]
    return ' '.join(meaningful_words)

for major in major_list:
    major['제품특성'] = preprocessing(major).apply(remove_stopwords)

In [ ]:
info = pd.DataFrame(df_merge.제품특성)
info.head(3)

In [ ]:
info['제품특성'] = preprocessing(info).apply(remove_stopwords)
info_corpus = info['제품특성'].values.tolist()
len(info_corpus)

In [ ]:
info_embeddings = model.encode(info_corpus)
corpuses = [major['제품특성'].values.tolist() for major in major_list]
len(corpuses)

In [ ]:
embeddings = []
for corpus in corpuses:
    embeddings.append(model.encode(corpus))

embeddings[0]

In [ ]:
def clustering(embeddings, corpus):
    num_clusters = 8
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(embeddings)
    cluster_assignment = clustering_model.labels_

    clustered_sentences = [[] for i in range(num_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(corpus[sentence_id])

    for i, cluster in enumerate(clustered_sentences):
        print('Cluster %d (%d)' % (i+1, len(cluster)))
        print(cluster)
        print('')

    return clustered_sentences

In [ ]:
clustering(embeddings[1], corpuses[1])

In [ ]:
clusters = clustering(info_embeddings, info_corpus)

In [ ]:
from konlpy.tag import Komoran, Okt, Kkma, Hannanum
from tqdm import tqdm
from collections import Counter
from wordcloud import WordCloud
from matplotlib import rc
%matplotlib inline
rc('font', family = 'AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

cnt = 1
for major in major_list:
    fig = plt.figure(figsize=(8, 4), facecolor= 'black')

    extractor = Hannanum()

    nouns = []

    for f in major['제품특성'].values.tolist():
        nouns.extend(extractor.nouns(f))

    count = Counter(nouns)
    words = dict(count.most_common())

    for i, (word, count) in enumerate(words.items()):
        if i > 5:
            break
        print(word, count)
        
    wc = WordCloud(
        font_path='/Users/a1r/Library/Fonts/NanumMyeongjo.ttf',
        width=2000,
        height=1000
    ).generate_from_frequencies(words)
    
    plt.title(f'[{cnt} Cluster]', color = 'white')
    plt.imshow(wc)
    plt.axis('off')

    plt.show()
    
    cnt += 1